In [17]:
import pandas as pd
from yahoo_finance import Share
from datetime import datetime, timedelta
import os
from datetime import date
import statsmodels.api as sm
from yahoofinancials import YahooFinancials

# Homework 2
Dandi Chen
Puja Subramanian
Brandan Whiteley
Trisha Paul

### Please run the functions at the bottom before running the program below (since they utilize the functions)

### Actual Program - Output include: cvs files that have data for both the companies and the bench mark as well as regression result

#### Query date from user, using functions

In [18]:
#get user input
limit = get_limit()
tickers_list = get_tickers(limit)
print('you are getting ', tickers_list)
start_date, end_date = get_year()
#this step is to get date formatted 
start = start_date[:10]
end = end_date[:10]
bench = get_benchmark()
rf = get_riskfree()

Enter the number of stock you want to query:2
Enter the stock you want to query (Ticker only):aapl
through
through 10
Enter the stock you want to query (Ticker only):msft
through
through 10
you are getting  ['AAPL', 'MSFT']
Date format: YYYYMMDD
Search Start year (Press enter to pass):20180101
Search end year (Press enter to pass):20190101
Enter the bench mark (options: Press 1 for SP500, Press 2 for Russell 3000, Press 3 for Nasdaq):1
Enter the risk free rate(please enter in percentage): 2


#### Organizing bench mark data to a dataframe that contains date and price (adjusted close)

In [24]:
## #get benchmark data
##using API to get adjusted price for bench mark
bench_date=[]
bench_price = []
bench_mark = YahooFinancials(bench)
bench_history = bench_mark.get_historical_price_data(start,end,'daily')
bench_his_all = bench_history.get(bench).get('prices')

for day in bench_his_all:
    bench_date.append(day.get('formatted_date'))
    bench_price.append(day.get('adjclose'))

#create data frame and write to csv
bench_df = pd.DataFrame(bench_date)
bench_df['price'] = bench_price
bench_df.rename(columns = {0:'date'})

bench_df.to_csv('benchmark_price.csv')

#### organizing company data to dataframe that contain date and price in a list
list format:

list = [[company name, company data in dataframe],[company name, company data in dataframe]]

In [20]:

comp_df_list=[]
#get company data
for ticker in tickers_list:
    pair = []
    company = YahooFinancials(ticker)
    history= company.get_historical_price_data(start, end,'daily')
    price_history_all =history.get(ticker).get('prices')
    date = []
    price = []
    for day in price_history_all:
        date.append(day.get('formatted_date'))
        price.append(day.get('adjclose'))
        
    df = pd.DataFrame(date)
    df['adj_price'] = price
    df.rename(columns = {0:'date'})
    filename = '{}.csv'.format(ticker)

    df.to_csv(filename)
    pair.append(ticker)
    pair.append(df)
    comp_df_list.append(pair)
    
        

#### get bench market return 

In [21]:
bench_df['shift'] = bench_df['price'].shift(1)
bench_df['ret'] = bench_df['price']/bench_df['shift']-1
bench_df['ret-rf']=bench_df['ret']-rf
bench_df = bench_df.fillna(0)
bench_return = bench_df['ret-rf']

#### get the company return and run regression

In [22]:
for comp in comp_df_list:
    compname = comp[0]
    df = comp[1]
    df['shift'] =  df['adj_price'].shift(1)
    df['ret'] =   df['adj_price']/  df['shift']-1
    df['ret-rf']=  df['ret']-rf
    df =df.fillna(0)
    df_return = df['ret-rf']
    
    #regression
    X=df_return
    Y=bench_return
    X1 = sm.add_constant(X)
    model = sm.OLS(Y, X1)
    res = model.fit()
    print(compname)
    print(res.summary())
    print('+++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++')
    print('+++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++')
    print('+++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++')
    print('+++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++')
    
    

AAPL
                            OLS Regression Results                            
Dep. Variable:                 ret-rf   R-squared:                       0.556
Model:                            OLS   Adj. R-squared:                  0.555
Method:                 Least Squares   F-statistic:                     312.4
Date:                Sat, 14 Dec 2019   Prob (F-statistic):           7.52e-46
Time:                        20:18:18   Log-Likelihood:                 882.86
No. Observations:                 251   AIC:                            -1762.
Df Residuals:                     249   BIC:                            -1755.
Df Model:                           1                                         
Covariance Type:            nonrobust                                         
                 coef    std err          t      P>|t|      [0.025      0.975]
------------------------------------------------------------------------------
const         -0.0112      0.001    -16.530    

/Library/Frameworks/Python.framework/Versions/3.7/lib/python3.7/site-packages/numpy/core/fromnumeric.py:2495: FutureWarning: Method .ptp is deprecated and will be removed in a future version. Use numpy.ptp instead.
  return ptp(axis=axis, out=out, **kwargs)


# The following are functions used in program above

#### GET TICKER AND CHECK CORRECTNESS
Tickers are checked by seeing if they have price data from early December 2019

In [10]:
#get ticker and check if that ticker is valid
def get_ticker():
    inputuser = False
    while inputuser == False:
        inputuser  = input('Enter the stock you want to query (Ticker only):')
        try:
            ticker = YahooFinancials(inputuser.upper())
            print('through')
            length = len(ticker.get_historical_price_data('2019-12-01', '2019-12-14', 'daily')\
                                            .get(inputuser.upper()).get('prices'))
            print('through',length)
            if length>0:
                return inputuser
            else:
                print('invalid, please try again')
                inputuser=False
        except:
            print('Error, please enter the correct ticker')
            inputuser = False
            
#control how many tickers the user is intend to query
def get_tickers(limit):
    ticker_list =[]
    for i in range(0, limit):
        ticker  = get_ticker()
        ticker_list.append(ticker.upper())
    
    return ticker_list
        

#### Get how many user wants to query

In [11]:
def get_limit():
    limit = False
    while limit == False:
        limit  = input('Enter the number of stock you want to query:')
        try:
            limit = int(limit)
            return limit
        except:
            print('The number you enter is not correct')
            limit = False
        
    

#### Get date range for the search. The date is limited to after 2010 for now. 
#### !!!! If User is querying two companys, one goes public before 2010 and one after, then the regression might have a problem 

In [12]:
#get dates
def get_year():
    right_year = False
    while right_year == False:
        print('Date format: YYYYMMDD')
        start_year = input('Search Start year (Press enter to pass):')
        end_year = input('Search end year (Press enter to pass):')
        right_year = date_validation(start_year, end_year)
        if right_year:
            start = str(pd.to_datetime(start_year,format='%Y-%m-%d'))
            end = str(pd.to_datetime(end_year,format='%Y-%m-%d'))
            return start, end
        else:
            right_year = False

#returns either False or valid date as pair [start,end]
def date_validation(start, end):
    current_date = int(date.today().strftime('%Y%m%d'))

    #see if user entered the right date
    try:
        start_test = pd.to_datetime(start,format='%Y%m%d')
        end_test = pd.to_datetime(end,format='%Y%m%d')
    #when the user enter the wrong content - entering anyting other than numbers 
    except:
        print('Please enter the right date')
        return False
    #check if reasonable begin &end date
    start = int(start)
    end = int(end)
    ### the number in the if statement can be changed to any date in %y%m%d format
    if start> current_date or start < 20100101 or end <20100101 or (end - start)<0 or end>current_date:
        print('Please check the year entry. The application do not support search prior to 2010.')
        return False
    else:
        return True

#### ask user which bench market they want. We are limiting to just SP500, Russel 3000 and Nasdaq composite for now

In [13]:
def get_benchmark():
    user_input = False
    while user_input == False:
        user_input = input('Enter the bench mark (options: Press 1 for SP500, Press 2 for Russell 3000, Press 3 for Nasdaq):')
        if user_input == '1':
            return '^GSPC'
        elif user_input == '2':
            return 'RUA'
        elif user_input == '3':
            return '.IXIC'
        else:
            print('Sorry, we do not currently support other tickers')
            user_input = False
                

#### Query the risk free rate from user. It's restricting user to enter anything from 0 percent to 10 percent. If user enters otherwise, error will show and ask again.

In [14]:
def get_riskfree():
#     current_date = int(datetime.strftime(datetime.now() - timedelta(1), '%Y-%m-%d'))
#     rf = 
    user_input = False
    while user_input == False:
        
        user_input = input('Enter the risk free rate(please enter in percentage): ')
        user_input = user_input.replace('%','')
        try:
            rf = float(user_input)
        except:
            user_input = False

        if rf<0 or rf>10:
            print('The risk free rate does not seem right, please enter again')
            user_input = False
        return rf/100